# Deep Learning Trading Bot


In [14]:
#!/usr/bin/env python3
"""
@author Alberto Bassi & Riccardo Tomada
"""
# Essential
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.express as px
import pandas as pd

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

# Neural Networks
from models import RNN_layer, LSTM_layer

# Progress bar
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from time import sleep



ImportError: cannot import name 'RNN_layer' from 'models' (/Users/albertobassi/Documents/NNDL/Sequential-Models/models.py)

## Dataset and Dataloader

# Define the Network

In [ ]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

### Set the random seed for reproducible results
torch.manual_seed(0)

### Initialize the two networks
x_dim = 3
h_dim = 3

net = LSTM_layer(x_dim, h_dim)

### Move to device
#net.to(device)

In [ ]:
# Check if the nets are working properly

x = torch.randn(1,x_dim)
h = torch.randn(1,h_dim)
C = torch.randn(1,h_dim)

out = net(x,h, C)
print(out[0])

# Training

## Save and load model